<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V6_SMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap plotly xgboost --quiet

In [2]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00


In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MJxCIqPSTQoAnA_Zt9u3FjjvXlWa74pzpC78XDuhQ9o/edit?usp=sharing')
worksheet = gsheets.worksheet("V6_expansion")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1200,NaN,NaN,NaN,0,180,1,80,0,45,20,45,25,17.677670,1,0.0,0.0,0.0,1.0,1
1,1252,NaN,NaN,NaN,0,180,1,80,0,55,20,55,35,12.738631,1,0.0,0.0,0.0,1.0,1
2,1178,NaN,NaN,NaN,0,180,0,81,0,10,10,10,0,NaN,0,0.0,0.0,0.0,1.0,1
3,1286,NaN,NaN,NaN,0,180,1,79,0,40,15,40,25,12.041595,1,0.0,0.0,0.0,1.0,1
4,1298,NaN,NaN,NaN,0,180,1,77,0,65,50,65,15,6.634888,1,0.0,1.0,0.0,1.0,1


In [7]:
external['呼吸_ADL低'] = ((external['使用呼吸輔具'] == 1) & (external['ADL_總分_max'] < 30)).astype(int)
external['高齡_住院多'] = ((external['預估年齡'] > 85) & (external['六個月內住院次數'] > 2)).astype(int)
external['ADL低_有管路'] = ((external['ADL_總分_max'] < 30) & (external['使用管路'] == 1)).astype(int)
external['DNR_呼吸'] = ((external['DNR_flag'] == 1) & (external['使用呼吸輔具'] == 1)).astype(int)
external['超級重症'] = (
    (external['DNR_flag'] == 1) &
    (external['使用呼吸輔具'] == 1) &
    (external['ADL_總分_max'] < 30) &
    (external['六個月內住院次數'] > 1)
).astype(int)
external["low_adl_no_var"] = ((external["ADL_std"].isnull()) & (external["ADL_總分_max"] < 5)).astype(int)

In [8]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,9827.0,1508.933652,3979.829302,1.0,1174.0,1234.0,1330.000000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,9827.0,0.412740,0.492352,0.0,0.0,0.0,1.000000,1.000000
觀察天數,9827.0,134.395543,63.203515,0.0,78.0,180.0,180.000000,180.000000
性別_is_male,9827.0,0.522336,0.499526,0.0,0.0,1.0,1.000000,1.000000
預估年齡,9827.0,79.366236,11.612847,0.0,73.0,81.0,88.000000,125.000000
DNR_flag,9827.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,9827.0,20.425359,28.851272,0.0,0.0,5.0,30.000000,100.000000


In [9]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


In [10]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UFokE2ATcNkejRy4PSCkcbCeYzrxOslhelzpd2OaPlc/edit?usp=sharing')
worksheet = gsheets.worksheet("V6")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,入家日期,結案日期,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1175,NaN,NaN,NaN,0,180,1,82,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,5
1,800,NaN,NaN,NaN,0,180,1,79,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,2
2,1262,NaN,NaN,NaN,0,180,0,77,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,3
3,1297,NaN,NaN,NaN,0,180,0,80,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,3
4,1240,NaN,NaN,NaN,0,180,1,82,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,4


In [11]:
df['呼吸_ADL低'] = ((df['使用呼吸輔具'] == 1) & (df['ADL_總分_max'] < 30)).astype(int)
df['高齡_住院多'] = ((df['預估年齡'] > 85) & (df['六個月內住院次數'] > 2)).astype(int)
df['ADL低_有管路'] = ((df['ADL_總分_max'] < 30) & (df['使用管路'] == 1)).astype(int)
df['DNR_呼吸'] = ((df['DNR_flag'] == 1) & (df['使用呼吸輔具'] == 1)).astype(int)
df['超級重症'] = (
    (df['DNR_flag'] == 1) &
    (df['使用呼吸輔具'] == 1) &
    (df['ADL_總分_max'] < 30) &
    (df['六個月內住院次數'] > 1)
).astype(int)
df["low_adl_no_var"] = ((df["ADL_std"].isnull()) & (df["ADL_總分_max"] < 5)).astype(int)

In [12]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,8782.0,1579.687657,4465.434171,6.0,1178.00,1243.0,1345.000000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
入家日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
結案日期,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,8782.0,0.477568,0.499525,0.0,0.00,0.0,1.000000,1.000000
觀察天數,8782.0,128.299362,64.349164,1.0,65.25,180.0,180.000000,180.000000
性別_is_male,8782.0,0.516283,0.499763,0.0,0.00,1.0,1.000000,1.000000
預估年齡,8782.0,78.948531,11.752762,0.0,72.00,81.0,87.000000,123.000000
DNR_flag,8782.0,0.000000,0.000000,0.0,0.00,0.0,0.000000,0.000000
ADL_總分_max,8782.0,24.066272,30.736311,0.0,0.00,10.0,40.000000,100.000000


In [13]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
性別_is_male,0,0.0000
觀察天數,0,0.0000
死亡標記,0,0.0000
預估年齡,0,0.0000
ADL_first_score,0,0.0000
ADL_總分_max,0,0.0000
DNR_flag,0,0.0000
ADL_diff,0,0.0000
ADL_明顯惡化,0,0.0000


📊 如何證明死亡組與對照組的變項分布無顯著差異？

#標準化平均差 (Standardized Mean Difference, SMD) 或 t-test P-value 建議以 SMD 為主（不受樣本數影響）

In [14]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# 確保 '死亡標記' 是二元
df = df[df['死亡標記'].isin([0, 1])].copy()

# 使用 df_missing_info.index 作為變數清單
variables = df_missing_info.index

# 定義計算 SMD 與 P-value 的函數
def compute_stats(df, var, group_col='死亡標記'):
    group0 = df[df[group_col] == 0][var].dropna()
    group1 = df[df[group_col] == 1][var].dropna()

    mean0 = group0.mean()
    mean1 = group1.mean()
    var0 = group0.var(ddof=1)
    var1 = group1.var(ddof=1)

    std_pooled = np.sqrt((var0 + var1) / 2)
    smd = (mean1 - mean0) / std_pooled if std_pooled > 0 else np.nan

    # 計算 P-value
    try:
        stat, pval = ttest_ind(group1, group0, equal_var=False)
    except Exception:
        pval = np.nan

    # NaN 原因說明
    if std_pooled == 0 or np.isnan(smd):
        reason = "兩組皆為常數或無變異"
    elif len(group0.unique()) == 1 and len(group1.unique()) == 1 and group0.unique()[0] == group1.unique()[0]:
        reason = "兩組數值完全一致"
    else:
        reason = ""

    return mean1, mean0, smd, pval, reason

# 計算所有變數的 SMD + P-value
smd_pval_results = []
for var in variables:
    mean1, mean0, smd, pval, reason = compute_stats(df, var)
    smd_pval_results.append({
        "變數": var,
        "死亡組平均": round(mean1, 3) if not np.isnan(mean1) else "NaN",
        "對照組平均": round(mean0, 3) if not np.isnan(mean0) else "NaN",
        "SMD": round(smd, 3) if not np.isnan(smd) else "NaN",
        "P-value": round(pval, 4) if not np.isnan(pval) else "NaN",
        "說明": reason
    })

# 轉為 DataFrame
smd_df = pd.DataFrame(smd_pval_results)

# 分為兩個表：無 NaN 和 含 NaN
smd_df_no_nan = smd_df[smd_df["SMD"] != "NaN"].reset_index(drop=True)
smd_df_with_nan = smd_df[smd_df["SMD"] == "NaN"].reset_index(drop=True)

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.11/dist-packages/scipy/_lib/deprecation.py:234: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return f(*args, **kwargs)


In [15]:
smd_df_no_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,H01_NUM,1388.639,1754.33,-0.084,0.0001,
1,性別_is_male,0.591,0.448,0.29,0.0,
2,觀察天數,71.742,180.0,-3.033,0.0,
3,預估年齡,80.366,77.652,0.233,0.0,
4,ADL_first_score,11.944,19.629,-0.305,0.0,
5,ADL_總分_max,15.547,31.854,-0.555,0.0,
6,ADL_diff,3.603,12.224,-0.523,0.0,
7,ADL_明顯惡化,0.112,0.368,-0.627,0.0,
8,ADL_last_score,15.547,31.854,-0.555,0.0,
9,呼吸_ADL低,0.239,0.244,-0.012,0.5889,


In [16]:
smd_df_with_nan

,變數,死亡組平均,對照組平均,SMD,P-value,說明
0,死亡標記,1.0,0.0,NaN,0.0,兩組皆為常數或無變異
1,DNR_flag,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
2,DNR_呼吸,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
3,超級重症,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
4,活動假牙,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
5,使用管路,1.0,1.0,NaN,NaN,兩組皆為常數或無變異
6,意識清醒_max,0.0,0.0,NaN,NaN,兩組皆為常數或無變異
7,入家日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
8,dbname,NaN,NaN,NaN,NaN,兩組皆為常數或無變異
9,結案日期,NaN,NaN,NaN,NaN,兩組皆為常數或無變異


In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency

# 建立年齡區間
age_bins = [65, 70, 75, 80, 85, 90, np.inf]
age_labels = ['65-69', '70-74', '75-79', '80-84', '85-89', '90+']
df['年齡區間'] = pd.cut(df['預估年齡'], bins=age_bins, labels=age_labels, right=False)

long_table_rows = []

# 修正後的分類變項
categorical_vars = {
    '年齡區間': age_labels,
    '性別_is_male': [0, 1],  # 0=女性, 1=男性
    'DNR_flag': [1],
    '意識清醒_max': [1],
    '使用管路': [1],
    '活動假牙': [1],
    '使用呼吸輔具': [1]
}

continuous_vars = [
    'ADL_總分_max',
    'ADL_first_score',
    '六個月內住院次數'
]

# 處理類別變項
for var, subgroups in categorical_vars.items():
    for val in subgroups:
        sub_df = df[df[var] == val]
        sub_df = sub_df[sub_df['死亡標記'].isin([0, 1])]

        died = sub_df[sub_df['死亡標記'] == 1]
        survived = sub_df[sub_df['死亡標記'] == 0]

        died_n = len(died)
        surv_n = len(survived)
        total_n = died_n + surv_n

        died_pct = died_n / total_n * 100 if total_n > 0 else np.nan
        surv_pct = surv_n / total_n * 100 if total_n > 0 else np.nan

        # 顯示標籤
        if var == '性別_is_male':
            subgroup_label = '男性' if val == 1 else '女性'
            var_label = '性別'
        elif var == 'DNR_flag':
            subgroup_label = '是'
            var_label = 'DNR'
        elif var == '意識清醒_max':
            subgroup_label = '清醒'
            var_label = '意識程度'
        elif var == '使用管路':
            subgroup_label = '使用'
            var_label = '管路'
        elif var == '活動假牙':
            subgroup_label = '使用'
            var_label = '假牙'
        elif var == '使用呼吸輔具':
            subgroup_label = '使用'
            var_label = '呼吸輔具'
        elif var == '年齡區間':
            subgroup_label = val
            var_label = '年齡'
        else:
            subgroup_label = str(val)
            var_label = var

        # 計算 P-value（類別變項使用卡方）
        try:
            ct = pd.crosstab(df[var] == val, df['死亡標記'])
            if ct.shape == (2, 2):
                _, pval, _, _ = chi2_contingency(ct)
                pval = f"{pval:.3f}"
            else:
                pval = "NA"
        except:
            pval = "NA"

        long_table_rows.append({
            '變項': var_label,
            '分類': subgroup_label,
            '死亡組 n': died_n,
            '死亡組 %': f"{died_pct:.1f}%" if not np.isnan(died_pct) else 'NA',
            '存活組 n': surv_n,
            '存活組 %': f"{surv_pct:.1f}%" if not np.isnan(surv_pct) else 'NA',
            'P-value': pval
        })

# 處理連續變項
for var in continuous_vars:
    died = df[df['死亡標記'] == 1][var].dropna()
    survived = df[df['死亡標記'] == 0][var].dropna()

    died_n = len(died)
    surv_n = len(survived)

    died_mean, died_std = died.mean(), died.std()
    surv_mean, surv_std = survived.mean(), survived.std()

    # p-value
    if died_n > 1 and surv_n > 1:
        _, pval = ttest_ind(died, survived, equal_var=False)
        pval = f"{pval:.3f}"
    else:
        pval = "NA"

    long_table_rows.append({
        '變項': var,
        '分類': '',
        '死亡組 n': died_n,
        '死亡組 %': f"{died_mean:.1f} ± {died_std:.1f}",
        '存活組 n': surv_n,
        '存活組 %': f"{surv_mean:.1f} ± {surv_std:.1f}",
        'P-value': pval
    })

# 整理成表格
long_summary_df = pd.DataFrame(long_table_rows)

In [18]:
long_summary_df

,變項,分類,死亡組 n,死亡組 %,存活組 n,存活組 %,P-value
0,年齡,65-69,330,46.2%,385,53.8%,0.392
1,年齡,70-74,419,37.9%,687,62.1%,0.000
2,年齡,75-79,500,40.4%,739,59.6%,0.000
3,年齡,80-84,809,50.2%,804,49.8%,0.035
4,年齡,85-89,873,54.3%,735,45.7%,0.000
5,年齡,90+,860,55.7%,683,44.3%,0.000
6,性別,女性,1715,40.4%,2533,59.6%,0.000
7,性別,男性,2479,54.7%,2055,45.3%,0.000
8,DNR,是,0,NA,0,NA,NA
9,意識程度,清醒,0,NA,0,NA,NA


In [19]:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 定義要分析的連續變數（解決 NameError）
features = ['ADL_總分_max', 'ADL_first_score', '六個月內住院次數']

# 準備欄位
group_cols = ['年齡區間', '性別_is_male']  # 使用 '性別_is_male' 代替 '性別'
output_rows = []

# 每個變項進行比較（明確指定 observed=True 以解決 FutureWarning）
for (age_group, gender_code), group_df in df.groupby(group_cols, observed=True):
    # 映射性別碼為文字標籤
    gender_label = '男性' if gender_code == 1 else '女性'

    row = {'年齡區間': age_group, '性別': gender_label, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

# 顯示結果
display(formatted_df)

,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,女性,246,115,0.467480,19.96 ± 26.77,36.87 ± 36.04,0.000,17.04 ± 24.95,22.79 ± 29.93,0.102,1.96 ± 2.56,3.43 ± 4.80,0.003
1,65-69,男性,469,215,0.458422,21.05 ± 28.26,37.66 ± 35.22,0.000,16.51 ± 24.88,24.06 ± 29.82,0.003,2.07 ± 1.62,4.11 ± 10.62,0.003
2,70-74,女性,414,129,0.311594,15.50 ± 21.24,37.18 ± 36.76,0.000,11.82 ± 19.31,23.68 ± 31.81,0.000,1.84 ± 1.50,4.22 ± 8.17,0.000
3,70-74,男性,692,290,0.419075,18.31 ± 23.77,37.34 ± 36.31,0.000,14.40 ± 21.16,23.25 ± 31.91,0.000,2.11 ± 2.10,3.32 ± 6.73,0.001
4,75-79,女性,561,178,0.317291,15.28 ± 24.00,34.19 ± 35.54,0.000,12.87 ± 21.82,21.16 ± 30.65,0.000,2.16 ± 2.55,3.90 ± 10.37,0.002
5,75-79,男性,678,322,0.474926,15.03 ± 23.23,35.98 ± 35.44,0.000,11.75 ± 20.05,22.30 ± 31.51,0.000,1.85 ± 1.47,6.03 ± 26.38,0.003
6,80-84,女性,823,321,0.390036,15.03 ± 21.98,32.35 ± 34.10,0.000,11.20 ± 18.94,19.58 ± 29.60,0.000,2.21 ± 2.40,4.14 ± 14.42,0.003
7,80-84,男性,790,488,0.617722,15.60 ± 22.74,33.10 ± 35.60,0.000,11.48 ± 19.21,21.79 ± 31.98,0.000,2.00 ± 2.08,3.07 ± 4.39,0.000
8,85-89,女性,934,419,0.448608,13.59 ± 21.50,26.90 ± 32.60,0.000,9.47 ± 17.75,14.42 ± 25.27,0.000,1.89 ± 1.37,3.13 ± 4.41,0.000
9,85-89,男性,674,454,0.673591,14.64 ± 23.41,26.66 ± 31.68,0.000,11.01 ± 20.12,15.18 ± 24.94,0.031,1.86 ± 1.65,3.21 ± 4.24,0.000


In [20]:
from scipy.stats import ttest_ind

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 準備欄位（修正：使用 '性別_is_male' 代替 '性別'）
group_cols = ['年齡區間', '性別_is_male']  # <- 這裡是關鍵修正
output_rows = []

# 每個變項進行比較
for (age_group, gender_code), group_df in df.groupby(group_cols):
    # 映射性別碼為文字標籤
    gender_label = '男性' if gender_code == 1 else '女性'

    row = {'年齡區間': age_group, '性別': gender_label, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:  # 假設 features 是你要比較的欄位列表（如 ['ADL_總分_max', '六個月內住院次數'] 等）
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

# 顯示結果（可選）
display(formatted_df)

/tmp/ipython-input-3029020933.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (age_group, gender_code), group_df in df.groupby(group_cols):


,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,女性,246,115,0.467480,19.96 ± 26.77,36.87 ± 36.04,0.000,17.04 ± 24.95,22.79 ± 29.93,0.102,1.96 ± 2.56,3.43 ± 4.80,0.003
1,65-69,男性,469,215,0.458422,21.05 ± 28.26,37.66 ± 35.22,0.000,16.51 ± 24.88,24.06 ± 29.82,0.003,2.07 ± 1.62,4.11 ± 10.62,0.003
2,70-74,女性,414,129,0.311594,15.50 ± 21.24,37.18 ± 36.76,0.000,11.82 ± 19.31,23.68 ± 31.81,0.000,1.84 ± 1.50,4.22 ± 8.17,0.000
3,70-74,男性,692,290,0.419075,18.31 ± 23.77,37.34 ± 36.31,0.000,14.40 ± 21.16,23.25 ± 31.91,0.000,2.11 ± 2.10,3.32 ± 6.73,0.001
4,75-79,女性,561,178,0.317291,15.28 ± 24.00,34.19 ± 35.54,0.000,12.87 ± 21.82,21.16 ± 30.65,0.000,2.16 ± 2.55,3.90 ± 10.37,0.002
5,75-79,男性,678,322,0.474926,15.03 ± 23.23,35.98 ± 35.44,0.000,11.75 ± 20.05,22.30 ± 31.51,0.000,1.85 ± 1.47,6.03 ± 26.38,0.003
6,80-84,女性,823,321,0.390036,15.03 ± 21.98,32.35 ± 34.10,0.000,11.20 ± 18.94,19.58 ± 29.60,0.000,2.21 ± 2.40,4.14 ± 14.42,0.003
7,80-84,男性,790,488,0.617722,15.60 ± 22.74,33.10 ± 35.60,0.000,11.48 ± 19.21,21.79 ± 31.98,0.000,2.00 ± 2.08,3.07 ± 4.39,0.000
8,85-89,女性,934,419,0.448608,13.59 ± 21.50,26.90 ± 32.60,0.000,9.47 ± 17.75,14.42 ± 25.27,0.000,1.89 ± 1.37,3.13 ± 4.41,0.000
9,85-89,男性,674,454,0.673591,14.64 ± 23.41,26.66 ± 31.68,0.000,11.01 ± 20.12,15.18 ± 24.94,0.031,1.86 ± 1.65,3.21 ± 4.24,0.000


In [21]:
from scipy.stats import ttest_ind

# 為了進行統計檢定與格式整理，我們將針對每個年齡區間與性別群體，
# 分別產生「死亡組」與「非死亡組」的平均、標準差與 p-value。

# 準備欄位
group_cols = ['年齡區間', '性別_is_male']
output_rows = []

# 每個變項進行比較
for (age_group, gender), group_df in df.groupby(group_cols):
    row = {'年齡區間': age_group, '性別': gender, '樣本數': len(group_df)}

    # 死亡/非死亡分開
    died = group_df[group_df['死亡標記'] == 1]
    survived = group_df[group_df['死亡標記'] == 0]

    row['死亡數'] = len(died)
    row['死亡率'] = len(died) / len(group_df) if len(group_df) > 0 else np.nan

    for feat in features:
        # 計算平均與標準差
        died_mean = died[feat].mean()
        died_std = died[feat].std()
        surv_mean = survived[feat].mean()
        surv_std = survived[feat].std()

        # t 檢定（排除 NaN）
        t_stat, p_val = ttest_ind(
            died[feat].dropna(), survived[feat].dropna(), equal_var=False
        ) if died[feat].dropna().size > 1 and survived[feat].dropna().size > 1 else (np.nan, np.nan)

        # 加入欄位（格式：mean ± std (p-value)）
        row[f'{feat}_死亡'] = f'{died_mean:.2f} ± {died_std:.2f}' if not np.isnan(died_mean) else 'NA'
        row[f'{feat}_存活'] = f'{surv_mean:.2f} ± {surv_std:.2f}' if not np.isnan(surv_mean) else 'NA'
        row[f'{feat}_p'] = f'{p_val:.3f}' if not np.isnan(p_val) else 'NA'

    output_rows.append(row)

# 整理成 DataFrame
formatted_df = pd.DataFrame(output_rows)

/tmp/ipython-input-730470339.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (age_group, gender), group_df in df.groupby(group_cols):


In [22]:
formatted_df

,年齡區間,性別,樣本數,死亡數,死亡率,ADL_總分_max_死亡,ADL_總分_max_存活,ADL_總分_max_p,ADL_first_score_死亡,ADL_first_score_存活,ADL_first_score_p,六個月內住院次數_死亡,六個月內住院次數_存活,六個月內住院次數_p
0,65-69,0,246,115,0.467480,19.96 ± 26.77,36.87 ± 36.04,0.000,17.04 ± 24.95,22.79 ± 29.93,0.102,1.96 ± 2.56,3.43 ± 4.80,0.003
1,65-69,1,469,215,0.458422,21.05 ± 28.26,37.66 ± 35.22,0.000,16.51 ± 24.88,24.06 ± 29.82,0.003,2.07 ± 1.62,4.11 ± 10.62,0.003
2,70-74,0,414,129,0.311594,15.50 ± 21.24,37.18 ± 36.76,0.000,11.82 ± 19.31,23.68 ± 31.81,0.000,1.84 ± 1.50,4.22 ± 8.17,0.000
3,70-74,1,692,290,0.419075,18.31 ± 23.77,37.34 ± 36.31,0.000,14.40 ± 21.16,23.25 ± 31.91,0.000,2.11 ± 2.10,3.32 ± 6.73,0.001
4,75-79,0,561,178,0.317291,15.28 ± 24.00,34.19 ± 35.54,0.000,12.87 ± 21.82,21.16 ± 30.65,0.000,2.16 ± 2.55,3.90 ± 10.37,0.002
5,75-79,1,678,322,0.474926,15.03 ± 23.23,35.98 ± 35.44,0.000,11.75 ± 20.05,22.30 ± 31.51,0.000,1.85 ± 1.47,6.03 ± 26.38,0.003
6,80-84,0,823,321,0.390036,15.03 ± 21.98,32.35 ± 34.10,0.000,11.20 ± 18.94,19.58 ± 29.60,0.000,2.21 ± 2.40,4.14 ± 14.42,0.003
7,80-84,1,790,488,0.617722,15.60 ± 22.74,33.10 ± 35.60,0.000,11.48 ± 19.21,21.79 ± 31.98,0.000,2.00 ± 2.08,3.07 ± 4.39,0.000
8,85-89,0,934,419,0.448608,13.59 ± 21.50,26.90 ± 32.60,0.000,9.47 ± 17.75,14.42 ± 25.27,0.000,1.89 ± 1.37,3.13 ± 4.41,0.000
9,85-89,1,674,454,0.673591,14.64 ± 23.41,26.66 ± 31.68,0.000,11.01 ± 20.12,15.18 ± 24.94,0.031,1.86 ± 1.65,3.21 ± 4.24,0.000


In [23]:
long_summary_df

,變項,分類,死亡組 n,死亡組 %,存活組 n,存活組 %,P-value
0,年齡,65-69,330,46.2%,385,53.8%,0.392
1,年齡,70-74,419,37.9%,687,62.1%,0.000
2,年齡,75-79,500,40.4%,739,59.6%,0.000
3,年齡,80-84,809,50.2%,804,49.8%,0.035
4,年齡,85-89,873,54.3%,735,45.7%,0.000
5,年齡,90+,860,55.7%,683,44.3%,0.000
6,性別,女性,1715,40.4%,2533,59.6%,0.000
7,性別,男性,2479,54.7%,2055,45.3%,0.000
8,DNR,是,0,NA,0,NA,NA
9,意識程度,清醒,0,NA,0,NA,NA
